In [19]:
import sys
import pandas as pd
import pyodbc as pyodbc
from sqlalchemy.dialects.mssql import pymssql
import sqlalchemy
import pymssql
import pymongo
import csv
import json
from config import cloudM,cloudMpassword,sqluser,sqlpass
from pymongo import MongoClient
from flask import Flask, jsonify, render_template
client = MongoClient()
cloudMClnt=MongoClient()
client = MongoClient('localhost', 27017)

In [20]:
cloudMClnt=MongoClient("mongodb+srv://"+ cloudM + ":"+ cloudMpassword + "@cluster0-omshy.mongodb.net/test?retryWrites=true&w=majority")

In [21]:
def cloudM_R():
    db=cloudMClnt['Aircraft']
    colmodels=db['models']
    colmodels2=db['models2']
    colmodels3=db['modelsold']
    colmodels4=db['solddetails']
    
    modelsdf = pd.DataFrame(list(colmodels.find()))
    modelsolddf = pd.DataFrame(list(colmodels3.find()))
    solddetailsdf = pd.DataFrame(list(colmodels4.find()))
    #modelsdf = pd.DataFrame(list(colmodels.find()))
    return modelsdf,modelsolddf,solddetailsdf

In [24]:
cloudmodelsdf,cloudsoldmodelsdf,cloudsolddetails = cloudM_R()

In [25]:
del cloudmodelsdf['_id']

res_fix = cloudmodelsdf[["ID", "MODEL_NO","DIMAID","WID","AIRLINE", "AIRCRAFT_TYPE","REGISTRATION",  "DESCRIPTION",  "SIZE", "PRICE",  "SHIPPING", "TAX",  "COMPANY", "DATEOFORDER",  "ORDEREDFROM", "PictureID",  "HangarClub"]]
cloudmodelsdf.head()
res_fix.head()

(res_fix.to_dict('records'))

[{'ID': 54,
  'MODEL_NO': 'WID11884',
  'DIMAID': nan,
  'WID': '11884',
  'AIRLINE': 'THAI AIRWAYS',
  'AIRCRAFT_TYPE': 'B747-400',
  'REGISTRATION': 'HS-TGJ',
  'DESCRIPTION': 'BigBird400 Mk.2 Thai Airways International B 747-4D7 1:400 - BB4-2005-XX - ID#11884; 2005s  Colors. Named  Haripunchai . With  Star Alliance Logo . With Cockpit Window Frames.HS-TGJ',
  'SIZE': '1:400',
  'PRICE': 19.0,
  'SHIPPING': 4.0,
  'TAX': 0.0,
  'COMPANY': 'BIGBIRD400 MK.2',
  'DATEOFORDER': Timestamp('2011-01-31 00:00:00'),
  'ORDEREDFROM': 'EBAY:JOHANCHAN2004',
  'PictureID': 54.0,
  'HangarClub': False},
 {'ID': 27,
  'MODEL_NO': 'PH4DLH346',
  'DIMAID': nan,
  'WID': '24315',
  'AIRLINE': 'LUFTHANSA',
  'AIRCRAFT_TYPE': 'A340-600',
  'REGISTRATION': 'D-AIHF',
  'DESCRIPTION': 'Phoenix Models Lufthansa A340-642 1:400 - PH4DLH346 - ID#24315 ; 1990s  Colors. Named  Lübeck . With  Star Alliance  Logo.  All metal construction with detailed scale landing gear .D-AIHF',
  'SIZE': '1:400',
  'PRICE': 20.4

In [6]:
del cloudsoldmodelsdf['_id']
cloudsoldmodelsdf.head()

,AIRCRAFT_TYPE,AIRLINE,COMPANY,DATEOFORDER,DESCRIPTION,DIMAID,HangarClub,ID,MODEL_NO,ORDEREDFROM,PRICE,PictureID,REGISTRATION,SHIPPING,SIZE,TAX,WID
0,B747-300,SINGAPORE AIRLINES,BIGBIRD,2011-02-02,BigBird Singapore Airlines B 747-312 1:500 - B...,None,False,1,BB5-2002-26A,WWW.EBAY.COM:chrism8036,27.00,0.0,9V-SKN,5.0,1:500,0.0,3694
1,AIRPORT:ACC,AIRPORT,HERPA,2011-02-13,HERPA AIRPORT ACCESSORIES X 519892 cars,None,False,2,HE519892,EBAY,25.00,0.0,ACCESSORIES,8.0,1:500,0.0,2452
2,A330-300,QATAR AIRWAYS,HERPA,2011-01-05,Herpa Qatar Airways A330-302 1:500 - 507998 - ...,None,False,4,HE507998,DAC andrews huber,37.00,4.0,A7-AEG,8.0,1:500,0.0,13856
3,A330-300,QATAR AIRWAYS,HERPA,2011-01-05,Herpa Qatar Airways A330-302 1:500 - 507523 - ...,None,False,5,HE507523,DAC andrews huber,37.00,0.0,A7-AED,8.0,1:500,0.0,12826
4,A330-200,QATAR AIRWAYS,DRAGON WINGS,2011-02-04,Dragon Wings Qatar Airways A330-202 1:400 - 55...,None,False,6,DRW55934,3000toys.com,20.95,0.0,A7-ACG,0.4,1:400,0.0,13473


In [7]:

del cloudsolddetails['_id']
cloudsolddetails.head()

,AircraftID,Buyer,EbayFee,ID,Insurance,Listing price,ListingFee,Net Recd,NetRecd,PaypalFee,SaleDate,Sale_Date,Shipping,month,price,profit_loss,shipping,tax,year
0,1,Ebay,5.40,1516,2.7,59.0,2.0,59.00,41.62,2.90,2018-06-17,2018-06-17,4.38,6,27.00,9.62,5.0,0.0,2018
1,2,EBAY,5.77,427,2.6,52.0,0.3,57.65,44.50,2.55,2014-02-15,2014-02-15,1.93,2,25.00,11.50,8.0,0.0,2014
2,4,EBAY,8.09,570,1.7,77.0,1.0,89.90,72.19,3.81,2014-09-22,2014-09-22,3.11,9,37.00,27.19,8.0,0.0,2014
3,5,Ebay,8.37,833,1.7,93.0,1.5,93.00,74.06,3.93,2015-06-28,2015-06-28,3.44,6,37.00,29.06,8.0,0.0,2015
4,6,Ebay,5.20,714,1.2,53.0,1.5,58.00,45.91,2.00,2015-02-22,2015-02-22,2.19,2,20.95,24.56,0.4,0.0,2015


In [8]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [9]:
def mongoR_I(exportdf,msdf,soldf):
    db=client['Aircraft']
    colmodels=db['models']
    colmodels2=db['models2']
    colmodelsales=db['modelsold']
    colsales=db['sales']
    cursor = colmodels.find() 
#for record in cursor: 
    colmodels.drop()
    
    colmodels2.drop()
    colmodelsales.drop()
    colsales.drop()
    #records = json.loads(exportdf.to_json(orient='records'))
    #records = json.loads(exportdf.T.to_json()).values()
    #db.models.insert_many(records)
    #db.insert_many(df.to_dict('records'))
    db.models.insert_many(exportdf.to_dict('records'))
    db.models2.insert_many(exportdf.to_dict('records'))
    db.modelsold.insert_many(msdf.to_dict('records'))
    db.sales.insert_many(soldf.to_dict('records'))

In [10]:
mongoR_I(cloudmodelsdf,cloudsoldmodelsdf,cloudsolddetails)

In [11]:
def elastic_update(exportdf,msdf,solddf):
    es.indices.delete(index='aircraft')
    es.indices.delete(index='aircraft_sales')
    es.indices.delete(index='solddetails')
    es.indices.create(index='aircraft', ignore=400)
    es.indices.create(index='aircraft_sales', ignore=400)
    es.indices.create(index='solddetails', ignore=400)
    INDEX="aircraft"
    TYPE= "models"
    from bson import json_util

    def rec_to_actions(export1df):
        #export1df["DATEOFORDER"]=export1df['DATEOFORDER'].dt.strftime('%Y-%m-%d')
        import json
        tmp = export1df.to_json(orient = "records")
        print(tmp)
    # Load each record into json format before bulk
        df_json= json.loads(tmp)
        for record in df_json: #export1df.to_dict(orient="records"):
            #print(record)
            yield ('{ "index" : { "_index" : "%s", "_type" : "%s" }}'% (INDEX, TYPE))
            yield (json.dumps(record, default=str))

    #from elasticsearch import Elasticsearch
    #e = Elasticsearch() # no args, connect to localhost:9200
    if not es.indices.exists(INDEX):
        raise RuntimeError('index does not exists, use `curl -X PUT "localhost:9200/%s"` and try again'%INDEX)

    r = es.bulk(rec_to_actions(exportdf)) # return a dict
    INDEX="aircraft_sales"
    TYPE= "sold"
    r1= es.bulk(rec_to_actions(msdf)) # return a dict
    INDEX="solddetails"
    TYPE= "solddet"
    r1= es.bulk(rec_to_actions(solddf)) # return a dict
    print(not r["errors"])

In [12]:
elastic_update(cloudmodelsdf,cloudsoldmodelsdf,cloudsolddetails)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[{"AIRCRAFT_TYPE":"B747-300","AIRLINE":"SINGAPORE AIRLINES","COMPANY":"BIGBIRD","DATEOFORDER":1296604800000,"DESCRIPTION":"BigBird Singapore Airlines B 747-312 1:500 - BB5-2002-26A - ID#3694 ;\"1980s\" Colors. Named \"Big Top\".9V-SKN","DIMAID":null,"HangarClub":false,"ID":1,"MODEL_NO":"BB5-2002-26A","ORDEREDFROM":"WWW.EBAY.COM:chrism8036","PRICE":27.0,"PictureID":0.0,"REGISTRATION":"9V-SKN","SHIPPING":5.0,"SIZE":"1:500","TAX":0.0,"WID":"3694"},{"AIRCRAFT_TYPE":"AIRPORT:ACC","AIRLINE":"AIRPORT","COMPANY":"HERPA","DATEOFORDER":1297555200000,"DESCRIPTION":"HERPA AIRPORT ACCESSORIES X 519892 cars","DIMAID":null,"HangarClub":false,"ID":2,"MODEL_NO":"HE519892","ORDEREDFROM":"EBAY","PRICE":25.0,"PictureID":0.0,"REGISTRATION":"ACCESSORIES","SHIPPING":8.0,"SIZE":"1:500","TAX":0.0,"WID":"2452"},{"AIRCRAFT_TYPE":"A330-300","AIRLINE":"QATAR AIRWAYS","COMPANY":"HERPA","DATEOFORDER":1294185600000,"DESCRIPTION":"Herpa Qatar Airways A330-302 1:500 - 507998 - ID#13856;\"15th Asian Games - Red\" Colors

[{"AircraftID":1,"Buyer":"Ebay","EbayFee":5.4,"ID":1516,"Insurance":2.7,"Listing price":59.0,"ListingFee":2.0,"Net Recd":59.0,"NetRecd":41.62,"PaypalFee":2.9,"SaleDate":1529193600000,"Sale_Date":1529193600000,"Shipping":4.38,"month":6,"price":27.0,"profit_loss":9.62,"shipping":5.0,"tax":0.0,"year":2018},{"AircraftID":2,"Buyer":"EBAY","EbayFee":5.77,"ID":427,"Insurance":2.6,"Listing price":52.0,"ListingFee":0.3,"Net Recd":57.65,"NetRecd":44.5,"PaypalFee":2.55,"SaleDate":1392422400000,"Sale_Date":1392422400000,"Shipping":1.93,"month":2,"price":25.0,"profit_loss":11.5,"shipping":8.0,"tax":0.0,"year":2014},{"AircraftID":4,"Buyer":"EBAY","EbayFee":8.09,"ID":570,"Insurance":1.7,"Listing price":77.0,"ListingFee":1.0,"Net Recd":89.9,"NetRecd":72.19,"PaypalFee":3.81,"SaleDate":1411344000000,"Sale_Date":1411344000000,"Shipping":3.11,"month":9,"price":37.0,"profit_loss":27.19,"shipping":8.0,"tax":0.0,"year":2014},{"AircraftID":5,"Buyer":"Ebay","EbayFee":8.37,"ID":833,"Insurance":1.7,"Listing pric

True


In [13]:
servername='yoga900'
username=sqluser
#pwd=input(" enter password : ")
pwd=sqlpass
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [14]:
def mongoR():
    db=client['Aircraft']
    colmodels=db['models']
    colmodels2=db['models2']
    colairsold=db['modelsold']
    colsdet=db["sales"]
    df = pd.DataFrame(list(colmodels.find()))
    df2 = pd.DataFrame(list(colairsold.find()))
    df3=pd.DataFrame(list(colsdet.find()))
    return df,df2,df3

In [15]:
mongodata,mongosold,mongosales=mongoR()

In [16]:
mongodata.head()
mongodata.fillna('')
mongodata['DIMAID'].fillna('',inplace=True)
mongodata['REGISTRATION'].fillna('',inplace=True)
mongodata['SHIPPING'].fillna(0,inplace=True)
mongodata['PRICE'].fillna(0,inplace=True)
mongodata['PictureID'].fillna('',inplace=True)
mongosold.fillna('')
mongosold['DIMAID'].fillna('',inplace=True)
mongosold['REGISTRATION'].fillna('',inplace=True)
mongosold['SHIPPING'].fillna(0,inplace=True)
mongosold['PRICE'].fillna(0,inplace=True)
mongosold['PictureID'].fillna('',inplace=True)

In [17]:
from sqlalchemy import create_engine, MetaData, Table, select
connection = pymssql.connect(host='yoga900',
                             user='arindam',
                             password='Airbusa350xwb',
                             database='Aircraft')
# Create a new record

#sql = "INSERT INTO dbo.aircraft (AIRCRAFT_TYPE, AIRLINE, COMPANY, DATEOFORDER, DESCRIPTION, DIMAID,HangarClub,ID,MODEL_NO,ORDEREDFROM,PRICE,PictureID,REGISTRATION,SHIPPING,SIZE,TAX,WID) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
sql = "INSERT INTO dbo.aircraft (ID,AIRCRAFT_TYPE, AIRLINE, COMPANY, DATEOFORDER, DESCRIPTION,HangarClub,MODEL_NO,ORDEREDFROM,PRICE,PictureID,REGISTRATION,SHIPPING,DIMAID,SIZE,TAX,WID) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s)"
cursor=connection.cursor()
cursor.execute("truncate table aircraft")
# Execute the query
for idx,data in mongodata.iterrows():
    cursor.execute(sql, (data["ID"],data["AIRCRAFT_TYPE"],data["AIRLINE"],data["COMPANY"],data["DATEOFORDER"],data["DESCRIPTION"],data["HangarClub"],data["MODEL_NO"],data["ORDEREDFROM"],data["PRICE"],data["PictureID"],data["REGISTRATION"],data["SHIPPING"],data["DIMAID"],data["SIZE"],data["TAX"],data["WID"]))

# the connection is not autocommited by default. So we must commit to save our changes.
connection.commit()

sql = "INSERT INTO dbo.aircraftsold (ID,AIRCRAFT_TYPE, AIRLINE, COMPANY, DATEOFORDER, DESCRIPTION,HangarClub,MODEL_NO,ORDEREDFROM,PRICE,PictureID,REGISTRATION,SHIPPING,DIMAID,SIZE,TAX,WID) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s)"
cursor=connection.cursor()
cursor.execute("truncate table aircraftsold")
# Execute the query
for idx,data in mongosold.iterrows():
    cursor.execute(sql, (data["ID"],data["AIRCRAFT_TYPE"],data["AIRLINE"],data["COMPANY"],data["DATEOFORDER"],data["DESCRIPTION"],data["HangarClub"],data["MODEL_NO"],data["ORDEREDFROM"],data["PRICE"],data["PictureID"],data["REGISTRATION"],data["SHIPPING"],data["DIMAID"],data["SIZE"],data["TAX"],data["WID"]))

# the connection is not autocommited by default. So we must commit to save our changes.
connection.commit()
#

sql = " insert into dbo.SoldDetails(ID,AircraftID,[Listing price],[Net Recd],SaleDate,ListingFee,EbayFee,PaypalFee,Shipping,Insurance,Buyer,NetRecd) VALUES (%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
print(sql)
cursor=connection.cursor()
cursor.execute("truncate table SoldDetails")
for idx,data in cloudsolddetails.iterrows():
    cursor.execute(sql, (data["ID"],data["AircraftID"],data["Listing price"],data["Net Recd"],data["SaleDate"],data["ListingFee"],data["EbayFee"],data["PaypalFee"],data["Shipping"],data["Insurance"],data["Buyer"],data["NetRecd"]))

# the connection is not autocommited by default. So we must commit to save our changes.
connection.commit()




 insert into dbo.SoldDetails(AircraftID,[Listing price],[Net Recd],SaleDate,ListingFee,EbayFee,PaypalFee,Shipping,Insurance,Buyer,NetRecd) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)


In [19]:
connection.commit()